In [24]:
from PIL import Image
from transformers import ViTFeatureExtractor, ViTModel
import torch

# 加载和确保图像为RGB
image_path = r"C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (12).png"
image = Image.open(image_path)
if image.mode != 'RGB':
    image = image.convert('RGB')

# 初始化特征提取器和模型
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# 预处理图像
inputs = feature_extractor(images=image, return_tensors='pt')

# 通过模型获取隐藏状态
with torch.no_grad():
    outputs = model(**inputs)
last_layer_hidden_states = outputs.last_hidden_state

# 应用全局平均池化
global_avg_pooled_features = torch.mean(last_layer_hidden_states.squeeze(0), dim=-1)

# 输出特征向量
global_avg_pooled_features


torch.Size([197])

In [18]:
global_avg_pooled_features

tensor([-0.0156])

In [26]:
import os
import glob
import pandas as pd
from PIL import Image
from transformers import ViTFeatureExtractor, ViTModel
import torch

# 定义输入和输出文件夹
input_folder = r"C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia"
output_csv = r"C:\Users\MYZ\PycharmProjects\毕设实验\超声图像\high-M.csv"

# 确保输出文件夹存在
if not os.path.exists(input_folder):
    os.makedirs(input_folder)

# 初始化特征提取器和模型
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# 创建一个空的列表来存储特征
all_features = []

# 遍历文件夹中的所有图像
for image_path in glob.glob(os.path.join(input_folder, '*.png')):  # 更改为匹配你的图像扩展名
    # 确保图像为RGB
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # 预处理图像
    inputs = feature_extractor(images=image, return_tensors='pt')

    # 通过模型获取隐藏状态
    with torch.no_grad():
        outputs = model(**inputs)
    last_layer_hidden_states = outputs.last_hidden_state

    # 应用全局平均池化
    global_avg_pooled_features = torch.mean(last_layer_hidden_states.squeeze(0), dim=-1)

    # 将特征向量展平并添加到列表
    flat_features = global_avg_pooled_features.numpy().flatten()
    all_features.extend(flat_features.tolist())

    print(f"Processed image: {image_path}")

# 创建DataFrame并保存为CSV
df = pd.DataFrame(all_features, columns=[f'feature_{i}' for i in range(len(all_features))])
df.insert(0, 'filename', [os.path.basename(image_path)] * len(all_features))

# 将DataFrame保存为CSV文件
df.to_csv(output_csv, index=False)

print("Features saved in CSV file: ", output_csv)


Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (10).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (100).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1000).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1001).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1002).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1003).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1004).png
Processed image: C:\Users\MYZ\Desktop\covid19-combo\COVID-19 Radiography Database\Viral Pneumonia\Viral Pneumonia (1005).png
Pro

ValueError: Shape of passed values is (263980, 1), indices imply (263980, 263980)